In [1]:
import rasterio
import geopandas as gpd
from shapely.geometry import box
from rasterio.warp import transform_bounds


冰川实测数据路径

In [ ]:
dsm_paths = [
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/24k/20231014/1_dsm/24k1014_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/24k/20240712/1_dsm/24k240712_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/KQGR/20230713/1_dsm/kqgr0713_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/KQGR/20230812/1_dsm/廓琼岗日230812_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/LAIGU/20230718/1_dsm/来古230718_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/LAIGU/20231017/1_dsm/laigu1017_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/P4/20230805/1_dsm/palung4_230805_dsm.tif",
        "data/ground-observation/glacier-station/国家青藏高原数据中心-无人机数据/P4/20231017/1_dsm/pal1016_dsm.tif"
            ]



In [3]:
# 实测点对应的RGI ID
rgi_map = {
    "24k": "RGI2000-v7.0-G-15-16601",
    "KQGR": "RGI2000-v7.0-G-13-61313",
    "LAIGU": "RGI2000-v7.0-G-15-14618",
    "P4": "RGI2000-v7.0-G-15-14585"
}

check data info

In [ ]:
records = []

for dsm_path in dsm_paths:
    try:
        with rasterio.open(dsm_path) as src:
            path_parts = dsm_path.split('/')
            glacier_name = path_parts[-4]
            data_date = path_parts[-3]
            bounds_4326 = transform_bounds(
                src.crs, "EPSG:4326",
                src.bounds.left, src.bounds.bottom,
                src.bounds.right, src.bounds.top
            )
            geom = box(*bounds_4326)
            records.append({
                "glacier": glacier_name,
                "rgi_id": rgi_map.get(glacier_name, None),
                "date": data_date,
                "resolution(m)": f"{src.res[0]:.3f}",
                "data_size": f"{src.height} × {src.width}",
                "geometry": geom
            })

    except Exception as e:
        print(f"Error reading file {dsm_path}: {e}")

gla_info_gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")
# gla_info_gdf.to_file("data/ground-observation/glacier-station/glacier_station_info.gpkg", driver="GPKG")
gla_info_gdf


,glacier,rgi_id,date,resolution(m),data_size,geometry
0,24k,RGI2000-v7.0-G-15-16601,20231014,0.035,35978 × 50313,"POLYGON ((95.72406 29.75652, 95.72406 29.76811..."
1,24k,RGI2000-v7.0-G-15-16601,20240712,0.033,38454 × 52468,"POLYGON ((95.72369 29.7564, 95.72369 29.76811,..."
2,KQGR,RGI2000-v7.0-G-13-61313,20230713,0.043,25112 × 36884,"POLYGON ((90.20499 29.85913, 90.20499 29.86917..."
3,KQGR,RGI2000-v7.0-G-13-61313,20230812,0.044,25586 × 35782,"POLYGON ((90.20506 29.8586, 90.20506 29.8692, ..."
4,LAIGU,RGI2000-v7.0-G-15-14618,20230718,0.269,12536 × 15929,"POLYGON ((96.81806 29.28251, 96.81806 29.31367..."
5,LAIGU,RGI2000-v7.0-G-15-14618,20231017,0.265,12006 × 17449,"POLYGON ((96.82136 29.2829, 96.82136 29.31243,..."
6,P4,RGI2000-v7.0-G-15-14585,20230805,0.056,46282 × 33462,"POLYGON ((96.94171 29.24191, 96.94171 29.26567..."
7,P4,RGI2000-v7.0-G-15-14585,20231017,0.054,50726 × 37174,"POLYGON ((96.94177 29.24047, 96.94177 29.26554..."
